# Sentiments

Calculate the Sentiments using twitter-XLM-roBERTa-base Model. [Model Card on Hugging Face](https://arxiv.org/pdf/2104.12250.pdf).[Corresponding Paper](https://arxiv.org/pdf/2104.12250.pdf).


In [2]:
from transformers import pipeline
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()


model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

sentiment_task("Putin ist ein Kriegsverbrecher")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


[{'label': 'negative', 'score': 0.9096924662590027}]

## Ukrainian Sentiments

In [3]:
df = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Language_min_dedupl/es.csv"
)

In [4]:
def calc_sentiments(text: str) -> dict[str, str | float]:
    sent = {"label": "error", "score": 0.0}
    try:
        sent = sentiment_task(text)[0]
    except Exception as e:
        print(e)
    return sent

In [12]:
df_cur

,Unnamed: 0,text,tweetcreatedts,tweetid,dupl,asd
12116400,14905011,Xbox head Phil Spencer said he supports Raven ...,2022-05-26 23:59:03.000000,1.529975e+18,[],13
12116401,14905012,@KyivIndependent That’s good news. The only wa...,2022-05-26 23:59:09.000000,1.529975e+18,[],13
12116402,14905014,When former #Oilers George Laraque sends a sho...,2022-05-26 23:59:15.000000,1.529975e+18,[],13
12116403,14905015,@miikegary Another reason why @Airbnb and also...,2022-05-26 23:59:18.000000,1.529975e+18,[],13
12116404,14905016,#Kissinger go fuck yourself. All decent people...,2022-05-26 23:59:25.000000,1.529976e+18,[],13
12116405,14905017,Cowards work hard to find pathetically pitiful...,2022-05-26 23:59:31.000000,1.529976e+18,[],13
12116406,14905018,"Despite #Russian heavy #Shelling, #Sievierodon...",2022-05-26 23:59:33.000000,1.529976e+18,[],13
12116407,14905019,"""#Russia warns the West against supplying Kiev...",2022-05-26 23:59:36.000000,1.529976e+18,[],13
12116408,14905020,"Not just wars, even pirates\n#Biden https://t....",2022-05-26 23:59:38.000000,1.529976e+18,[],13
12116409,14905021,@Soniapango @MullicaMichelle @ACLU @GOP @SDNYn...,2022-05-26 23:59:49.000000,1.529976e+18,[],13


In [15]:
n = 0
for i in tqdm(range(0,df.shape[0], int(df.shape[0]/100))):
    df_cur = df.iloc[i:i+int(df.shape[0]/100)].copy()
    df_cur["sentiment"] = df_cur["text"].progress_apply(calc_sentiments)
    df_cur.to_csv(f"/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Sentiments/english/en_{n}.csv")
    n += 1


  1%|          | 1/101 [1:34:09<156:55:55, 5649.55s/it]

In [5]:
df["sentiment"] = df["text"].progress_apply(calc_sentiments)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/1002517 [00:00<?, ?it/s]

  0%|          | 1163/1002517 [01:38<24:50:39, 11.20it/s]

In [65]:
df["sentiment_score"] = df["sentiment"].apply(lambda x: x["score"])
df["sentiment_label"] = df["sentiment"].apply(lambda x: x["label"])

In [67]:
df.to_csv("/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Sentiments/fr.csv")